In [11]:
from arraytainers import Arraytainer, Jaxtainer
import numpy as np
from numpy import array
import jax
import jax.numpy as jnp
import functools
from jax.numpy import DeviceArray

In [2]:
type(jnp.array([1,2]).copy())

numpy.ndarray

In [3]:
a = Arraytainer({'mean': jnp.array([1.]), 'chol_diag': jnp.array([1]), 'chol_lowerdiag': jnp.array([0])})#{'a': {'c':[np.random.rand(6,3,5,2)], 'd':np.random.rand(6,3,5,2)}, 'b': np.random.rand(3,3,5,5), 'e':2}
a

Arraytainer({'mean': array([1.], dtype=float32), 'chol_diag': array([1], dtype=int32), 'chol_lowerdiag': array([0], dtype=int32)})

In [10]:
a = Jaxtainer({'mean': np.array([1.]), 'chol_diag': np.array([1]), 'chol_lowerdiag': np.array([0])})#{'a': {'c':[np.random.rand(6,3,5,2)], 'd':np.random.rand(6,3,5,2)}, 'b': np.random.rand(3,3,5,5), 'e':2}
a

Jaxtainer({'mean': DeviceArray([1.], dtype=float32), 'chol_diag': DeviceArray([1], dtype=int32), 'chol_lowerdiag': DeviceArray([0], dtype=int32)})

In [5]:
a = {'a': np.random.rand(4,3,5,5), 'b': np.random.rand(3,3,5,5)}
A = Jaxtainer(a)
# A.reshape(A.shape[:-2], 25)

In [6]:
A.shape

Jaxtainer({'a': DeviceArray([4, 3, 5, 5], dtype=int32), 'b': DeviceArray([3, 3, 5, 5], dtype=int32)})

In [7]:
a = {'a': np.random.rand(4,3,5), 'b': np.random.rand(3,4,5,5)}
A = Jaxtainer(a)
A.reshape(12, A.shape[2:]).shape

Jaxtainer({'a': DeviceArray([12,  5], dtype=int32), 'b': DeviceArray([12,  5,  5], dtype=int32)})

In [8]:
Jaxtainer.from_array(A.flatten(), A.shape).shape

Jaxtainer({'a': DeviceArray([4, 3, 5], dtype=int32), 'b': DeviceArray([3, 4, 5, 5], dtype=int32)})

In [9]:
np.array(2)[None]

array([2])